## DFO Nutrient Comparison

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import netCDF4 as nc
import datetime as dt
from salishsea_tools import evaltools as et, viz_tools
import gsw
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import matplotlib.dates as mdates
import cmocean as cmo
import scipy.interpolate as sinterp
import pytz
import warnings

mpl.rc('xtick', labelsize=10)
mpl.rc('ytick', labelsize=10)
mpl.rc('legend', fontsize=10)
mpl.rc('axes', titlesize=10)
mpl.rc('axes', labelsize=10)
mpl.rc('figure', titlesize=10)
mpl.rc('font', size=10)
%matplotlib inline

In [2]:
PATH= '/data/eolson/results/MEOPAR/SS36runs/linkHC201812/'
#PATH= '/results2/SalishSea/hindcast/'
start_date = dt.datetime(2015,1,1)
end_date = dt.datetime(2018,1,1)
flen=1
namfmt='nowcast'
#varmap={'N':'nitrate','Si':'silicon','Ammonium':'ammonium'}
filemap={'nitrate':'ptrc_T','silicon':'ptrc_T','ammonium':'ptrc_T','diatoms':'ptrc_T','ciliates':'ptrc_T','flagellates':'ptrc_T','vosaline':'grid_T','votemper':'grid_T'}
#gridmap={'nitrate':'tmask','silicon':'tmask','ammonium':'tmask'}
fdict={'ptrc_T':1,'grid_T':1}


In [3]:
datelims=(dt.datetime(1900,1,1),dt.datetime(2100,1,1))
start_date=datelims[0]
end_date=datelims[1]    

f0 = pd.read_excel('/ocean/eolson/MEOPAR/obs/Hakai/Dosser20180911/2018-09-11_144804_HakaiData_nutrients.xlsx',
                 sheet_name = 'Hakai Data')
f0.drop(['ACTION','Lat', 'Long', 'Collection Method', 'Installed', 'Lab Technician', 'NH4+', 'NO2+NO3 (ug/L)',
       'no2_no3_units', 'TP', 'TDP', 'TN', 'TDN', 'SRP', 'Project Specific ID', 'Hakai ID', 'Source',
       'po4pfilt', 'no3nfilt', 'po4punfl', 'no3nunfl', 'nh4nunfl', 'NH4+ Flag',
       'TP FLag', 'TDP FLag', 'TN Flag', 'TDN FLag','Volume (ml)',
       'SRP Flag', 'PO4 Flag', 'po4pfilt_flag', 'no3nfilt_flag','Preserved', 'Analyzed',
       'po4punfl_flag', 'no3nunfl_flag', 'nh4nunfl_flag', 'Analyzing Lab', 'Sample Status',
       'Quality Level', 'Comments', 'Quality Log'], axis = 1, inplace = True)
dts0=[pytz.timezone('Canada/Pacific').localize(i).astimezone(pytz.utc).replace(tzinfo=None)
        for i in f0['Collected']]
f0['dtUTC']=dts0

In [4]:
fc = pd.read_csv('/ocean/eolson/MEOPAR/obs/Hakai/Dosser20180911/ctd-bulk-1536702711696.csv',
                usecols=['Cast PK','Cruise','Station', 'Drop number','Start time', 'Bottom time',
                         'Latitude', 'Longitude', 'Depth (m)', 'Temperature (deg C)', 'Temperature flag', 'Pressure (dbar)',
                         'Pressure flag', 'PAR', 'PAR flag', 'Fluorometry Chlorophyll (ug/L)', 'Fluorometry Chlorophyll flag',
                         'Turbidity (FTU)', 'Turbidity flag',
                         'Salinity (PSU)', 'Salinity flag'],
                dtype={'Drop number':np.float64,'PAR flag':str,'Fluorometry Chlorophyll flag':str},na_values=('null','-9.99e-29'))

## fix apparent typos:
# reversed lats and lons
iii=fc['Latitude']>90
lons=-1*fc.loc[iii,'Latitude'].values
lats=-1*fc.loc[iii,'Longitude'].values
fc.loc[iii,'Longitude']=lons
fc.loc[iii,'Latitude']=lats

# remove data with missing lats and lons
nans=fc.loc[(fc['Latitude'].isnull())|(fc['Longitude'].isnull())]
fc=fc.drop(nans.index)
# apparently bad lats/lons
QU16bad=fc.loc[(fc['Station']=='QU16')&(fc['Latitude']>50.3)]
fc=fc.drop(QU16bad.index)
QU36bad=fc.loc[(fc['Station']=='QU36')&(fc['Latitude']>50.2)]
fc=fc.drop(QU36bad.index)
QU37bad=fc.loc[(fc['Station']=='QU37')&(fc['Longitude']<-125.1)]
fc=fc.drop(QU37bad.index)
QU38bad=fc.loc[(fc['Station']=='QU38')&(fc['Longitude']>-125.2)]
fc=fc.drop(QU38bad.index)
QU5bad=fc.loc[(fc['Station']=='QU5')&(fc['Longitude']>-125.18)]
fc=fc.drop(QU5bad.index)

In [5]:
list(fc)

['Cast PK',
 'Cruise',
 'Station',
 'Drop number',
 'Start time',
 'Bottom time',
 'Latitude',
 'Longitude',
 'Depth (m)',
 'Temperature (deg C)',
 'Temperature flag',
 'Pressure (dbar)',
 'Pressure flag',
 'PAR',
 'PAR flag',
 'Fluorometry Chlorophyll (ug/L)',
 'Fluorometry Chlorophyll flag',
 'Turbidity (FTU)',
 'Turbidity flag',
 'Salinity (PSU)',
 'Salinity flag']

In [6]:
fc.loc[(fc['Temperature (deg C)']==0)&(fc['Salinity (PSU)']==0)]

,Cast PK,Cruise,Station,Drop number,Start time,Bottom time,Latitude,Longitude,Depth (m),Temperature (deg C),...,Pressure (dbar),Pressure flag,PAR,PAR flag,Fluorometry Chlorophyll (ug/L),Fluorometry Chlorophyll flag,Turbidity (FTU),Turbidity flag,Salinity (PSU),Salinity flag
115143,836,QOM A,QU39,1.0,2015-06-30 07:36:15.000,2015-06-30 07:45:57.000,50.030668,-125.099217,0.0,0.0,...,270,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
115144,836,QOM A,QU39,1.0,2015-06-30 07:36:15.000,2015-06-30 07:45:57.000,50.030668,-125.099217,0.0,0.0,...,271,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
115145,836,QOM A,QU39,1.0,2015-06-30 07:36:15.000,2015-06-30 07:45:57.000,50.030668,-125.099217,0.0,0.0,...,272,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
115146,836,QOM A,QU39,1.0,2015-06-30 07:36:15.000,2015-06-30 07:45:57.000,50.030668,-125.099217,0.0,0.0,...,273,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
115838,840,QOM A,QU39,1.0,2015-07-07 07:58:36.000,2015-07-07 08:08:29.000,50.030668,-125.099217,0.0,0.0,...,267,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
115839,840,QOM A,QU39,1.0,2015-07-07 07:58:36.000,2015-07-07 08:08:29.000,50.030668,-125.099217,0.0,0.0,...,268,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
120090,878,QOM A,QU39,1.0,2015-07-28 08:42:47.000,2015-07-28 08:53:39.000,50.030668,-125.099217,0.0,0.0,...,238,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
120091,878,QOM A,QU39,1.0,2015-07-28 08:42:47.000,2015-07-28 08:53:39.000,50.030668,-125.099217,0.0,0.0,...,239,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
122892,896,QOM A,QU39,1.0,2015-08-12 08:48:15.000,2015-08-12 09:01:08.000,50.030668,-125.099217,0.0,0.0,...,239,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
129975,956,QOM A,QU39,1.0,2015-09-02 09:10:27.000,2015-09-02 09:22:24.000,50.030668,-125.099217,0.0,0.0,...,255,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN


In [7]:
fc.loc[(fc['Salinity (PSU)']<0)]

,Cast PK,Cruise,Station,Drop number,Start time,Bottom time,Latitude,Longitude,Depth (m),Temperature (deg C),...,Pressure (dbar),Pressure flag,PAR,PAR flag,Fluorometry Chlorophyll (ug/L),Fluorometry Chlorophyll flag,Turbidity (FTU),Turbidity flag,Salinity (PSU),Salinity flag


In [ ]:
# remove data with suspicious 0 temperature and salinity
iind=(fc['Temperature (deg C)']==0)&(fc['Salinity (PSU)']==0)
fc.loc[iind,['Temperature (deg C)', 'Pressure (dbar)', 'PAR', 'Fluorometry Chlorophyll (ug/L)', 'Turbidity (FTU)', 'Salinity (PSU)']]=np.nan

fc['dt']=[dt.datetime.strptime(i.split('.')[0],'%Y-%m-%d %H:%M:%S') for i in fc['Start time']]
dts=[pytz.timezone('Canada/Pacific').localize(dt.datetime.strptime(i.split('.')[0],'%Y-%m-%d %H:%M:%S')).astimezone(pytz.utc).replace(tzinfo=None)
        for i in fc['Start time']]
fc['dtUTC']=dts

dloc=[dt.datetime(i.year,i.month,i.day) for i in fc['dt']]
fc['dloc']=dloc

In [ ]:
fcS=fc.loc[:,['Latitude','Longitude']].groupby([fc['Station'],fc['dloc']]).mean().reset_index()

f0['Station']=f0['Site ID']
#f0['dt']=[dt.datetime.strptime(i,'%Y-%m-%d %H:%M:%S') for i in f0['Collected']]
dloc0=[dt.datetime(i.year,i.month,i.day) for i in f0['Collected']]
f0['dloc']=dloc0

In [ ]:
f0.drop(['no','event_pk','Sampling Bout','Gather Lat','Gather Long','Filter Type'],axis=1,inplace=True)

In [ ]:
f0.loc[f0['Pressure Transducer Depth (m)']<1.7].head()

In [ ]:
plt.hist(f0.loc[~np.isnan(f0['Line Out Depth']),['Line Out Depth']].values,250);

In [ ]:
np.min(f0['Pressure Transducer Depth (m)'])

In [ ]:
fc.drop(['Cast PK','Drop number','Start time','Bottom time'],axis=1,inplace=True)

In [ ]:
fc.head()

In [ ]:
fdata=f0.merge(fcS,how='left')

In [ ]:
fdata.head()

In [ ]:
set(list(f0)).intersection(set(list(fcS)))

In [ ]:
set(list(f0)).intersection(set(list(fc)))

In [ ]:
fc.loc[(np.isnan(fc['Depth (m)']))&(~np.isnan(fc['Pressure (dbar)']))]

In [ ]:
fc.loc[(~np.isnan(fc['Depth (m)']))&(np.isnan(fc['Pressure (dbar)']))]

In [ ]:

fdata['SA']=np.nan
fdata['CT']=np.nan
fdata['pZ']=np.nan

In [ ]:
df2=fdata.copy(deep=True)

In [ ]:
zthresh=1.5
print("Warning: CTD depths may vary from bottle depths by up to ",str(zthresh)," m.")
for i, row in df2.iterrows():
    idf=fc.loc[(fc.Station==row['Station'])&(fc.dloc==row['dloc'])&\
               ((np.abs(fc['Depth (m)']-row['Pressure Transducer Depth (m)'])<zthresh)|(np.abs(fc['Depth (m)']-row['Line Out Depth'])<zthresh))]
    #print(row['dtUTC'],row['Station'],row['Line Out Depth'],row['Pressure Transducer Depth (m)'])
    #print(idf.loc[:,['Station','dtUTC','Depth (m)','Pressure (dbar)','Temperature (deg C)','Salinity (PSU)']])
    if len(idf)>0:
        zrow=row['Pressure Transducer Depth (m)'] if ~np.isnan(row['Pressure Transducer Depth (m)']) else row['Line Out Depth']
        #print(zrow)
        zdifmin=np.min([np.abs(ii-zrow) for ii in idf['Depth (m)']])
        # if there are multiple minimum distance rows, just take the first
        idfZ=idf.loc[np.abs(idf['Depth (m)']-zrow)==zdifmin]
        #print(idfZ.loc[:,['Station','dtUTC','Depth (m)','Pressure (dbar)','Temperature (deg C)','Salinity (PSU)']])
        sal=gsw.SA_from_SP(idfZ['Salinity (PSU)'].values[0],idfZ['Pressure (dbar)'].values[0],idfZ['Longitude'].values[0],idfZ['Latitude'].values[0])
        tem=gsw.CT_from_t(sal,idfZ['Temperature (deg C)'].values[0],idfZ['Pressure (dbar)'].values[0])
        #print('sal:',sal,'tem:',tem,'z:',idfZ['Depth (m)'].values[0])
        fdata.at[i,'SA']=sal
        fdata.at[i,'CT']=tem
        fdata.at[i,'pZ']=idfZ['Depth (m)'].values[0]
    #print('-------------------------------')

In [ ]:
PATH= '/results2/SalishSea/hindcast/'
start_date = dt.datetime(2015,1,1)
end_date = dt.datetime(2018,1,1)
flen=1
namfmt='nowcast'
#varmap={'N':'nitrate','Si':'silicon','Ammonium':'ammonium'}
filemap={'nitrate':'ptrc_T','silicon':'ptrc_T','ammonium':'ptrc_T','diatoms':'ptrc_T','ciliates':'ptrc_T','flagellates':'ptrc_T','vosaline':'grid_T','votemper':'grid_T'}
#gridmap={'nitrate':'tmask','silicon':'tmask','ammonium':'tmask'}
fdict={'ptrc_T':1,'grid_T':1}
#df1=et.loadDFO()
#df1.head()
fdata['AbsSal']=fdata['SA']
fdata['ConsT']=fdata['CT']
fdata['Lat']=fdata['Latitude']
fdata['Lon']=fdata['Longitude']
df1=fdata
df1['NO23']=df1['NO2+NO3 (uM)']
df1['Si']=df1['SiO2']

In [ ]:
list(fdata)

In [ ]:
plt.plot(df1['SA'],df1['NO23'],'k.')

In [ ]:
plt.plot(df1['SA'],df1['CT'],'k.')

In [ ]:
df1.loc[(~np.isnan(df1.SA)),['SA']].count()

In [ ]:
df1.loc[(~np.isnan(df1.SA))&(np.isnan(df1.NO23)),['SA']].count()

In [ ]:
df1.loc[(np.isnan(df1.SA))&(~np.isnan(df1.NO23)),['NO23']].count()

In [ ]:
df1.loc[~np.isnan(df1['CT']),['CT']].count()

In [ ]:
df1.loc[(~np.isnan(df1['CT']))&(np.isnan(df1.NO23)),['CT']].count()

In [ ]:
df1.loc[(np.isnan(df1['CT']))&(~np.isnan(df1.NO23)),['NO23']].count()

In [ ]:
df1.loc[(~np.isnan(df1.NO23)),['NO23']].count()

In [ ]:
print('% of N values without S:')
print(df1.loc[(np.isnan(df1.SA))&(~np.isnan(df1.NO23)),['NO23']].count().values/df1.loc[(~np.isnan(df1.NO23)),['NO23']].count().values*100)

In [ ]:
list(df1)

In [ ]:
# use only matched locs. 
df2=df1.loc[(df1.NO23>=0)|(df1.Si>=0),['Site ID', 'Line Out Depth', 'Pressure Transducer Depth (m)', 'dtUTC', 'Station', 'Lat', 'Lon', 'SA', 'CT', 'pZ', 'AbsSal', 'ConsT']].copy(deep=True)

In [ ]:
df2['Z']=df2['pZ']

In [ ]:
dataPSF=et.matchData(df2,filemap, fdict, start_date, end_date, namfmt, PATH, flen)

In [ ]:
def rotmap(lon,lat,lon0,lat0,phi):
    # rotate around point (pick one near center of domain)
    # phi in degrees
    # first scale lats to match and center around lat0,lon0:
    lon1=(lon-lon0)*np.cos(lat0*np.pi/180)
    lat1=lat-lat0
    # now rotate:
    lon2=lon1*np.cos(phi*np.pi/180)-lat1*np.sin(phi*np.pi/180)
    lat2=lon1*np.sin(phi*np.pi/180)+lat1*np.cos(phi*np.pi/180)
    return lon2,lat2

In [ ]:
cm1=plt.get_cmap('PuBuGn')
#cm1=cmo.cm.matter
theta=-30
lon0=-123.9
lat0=49.3
with nc.Dataset('/data/eolson/results/MEOPAR/NEMO-forcing-new/grid/bathymetry_201702.nc') as bathy:
    bathylon=np.copy(bathy.variables['nav_lon'][:,:])
    bathylat=np.copy(bathy.variables['nav_lat'][:,:])
    bathyZ=np.copy(bathy.variables['Bathymetry'][:,:])
blon,blat=rotmap(bathylon,bathylat,lon0,lat0,theta)

In [ ]:
fig = plt.figure(figsize = (7.5,4.5))
gs1=gridspec.GridSpec(2,4,left=.08,right=.98,bottom=.015,top=.94,
                      wspace=.34,hspace=.34,height_ratios=[1,1],width_ratios=[1,1,1,.1])
ax2015N=fig.add_subplot(gs1[0,0])
ax2016N = fig.add_subplot(gs1[0,1])
ax2017N = fig.add_subplot(gs1[0,2])
ax2015Si= fig.add_subplot(gs1[1,0])
ax2016Si= fig.add_subplot(gs1[1,1])
ax2017Si= fig.add_subplot(gs1[1,2])
gscb=gridspec.GridSpecFromSubplotSpec(5,1,subplot_spec=gs1[:,3])
axcb = fig.add_subplot(gscb[1:-1])

for ax in (ax2015N,ax2016N,ax2017N):
    ax.plot((0,36),(0,36),'k-',alpha=.2)
for ax in (ax2015Si,ax2016Si,ax2017Si):
    ax.plot((0,25),(0,25),'k-',alpha=.2)

vm0=-150
args={'marker':'.','s':1,}
dataPSF2015=dataPSF.loc[(dataPSF.dtUTC>=dt.datetime(2015,1,1))&(dataPSF.dtUTC<dt.datetime(2016,1,1))]
ps=et.varvarScatter(ax2015N,dataPSF2015,'AbsSal','mod_vosaline','Z',vmin=vm0,vmax=450,cm=cm1,args=args)
dataPSF2016=dataPSF.loc[(dataPSF.dtUTC>=dt.datetime(2016,1,1))&(dataPSF.dtUTC<dt.datetime(2017,1,1))]
ps=et.varvarScatter(ax2016N,dataPSF2016,'AbsSal','mod_vosaline','Z',vmin=vm0,vmax=450,cm=cm1,args=args)
dataPSF2017=dataPSF.loc[(dataPSF.dtUTC>=dt.datetime(2017,1,1))&(dataPSF.dtUTC<dt.datetime(2018,1,1))]
ps=et.varvarScatter(ax2017N,dataPSF2017,'AbsSal','mod_vosaline','Z',vmin=vm0,vmax=450,cm=cm1,args=args)
ps=et.varvarScatter(ax2015Si,dataPSF2015,'ConsT','mod_votemper','Z',vmin=vm0,vmax=450,cm=cm1,args=args)
ps=et.varvarScatter(ax2016Si,dataPSF2016,'ConsT','mod_votemper','Z',vmin=vm0,vmax=450,cm=cm1,args=args)
ps=et.varvarScatter(ax2017Si,dataPSF2017,'ConsT','mod_votemper','Z',vmin=vm0,vmax=450,cm=cm1,args=args)

cb=fig.colorbar(ps,cax=axcb,boundaries=np.linspace(0,450,46))
cb.set_label('Depth (m)')

ntick=np.arange(0,36,10)
ntickl=[str(i) for i in ntick]
for ax in (ax2015N,ax2016N,ax2017N):
    ax.set_xlim((0,36))
    ax.set_ylim((0,36))
    ax.set_xticks(ntick)
    ax.set_xticklabels(ntickl)
    ax.set_yticks(ntick)
    ax.set_yticklabels(ntickl)
    
stick=np.arange(0,25,10)
stickl=[str(i) for i in stick]
for ax in (ax2015Si,ax2016Si,ax2017Si):
    ax.set_xlim((0,25))
    ax.set_ylim((0,25))
    ax.set_xticks(stick)
    ax.set_xticklabels(stickl)
    ax.set_yticks(stick)
    ax.set_yticklabels(stickl)
    
for ax in (ax2015N,ax2016N,ax2017N,ax2015Si,ax2016Si,ax2017Si):
    ax.set_aspect(1, adjustable='box')
    #ax.set_xlabel('Observed')
    
ax2015N.set_ylabel('Modeled',fontsize=12)
ax2015Si.set_ylabel('Modeled',fontsize=12)
ax2015Si.set_ylabel('Modeled',fontsize=12)
ax2015Si.set_xlabel('Observed',fontsize=12)
ax2016Si.set_xlabel('Observed',fontsize=12)
ax2017Si.set_xlabel('Observed',fontsize=12)

ax2015N.annotate('2015',xy=[.5,1.18],xycoords='axes fraction',ha='center',fontsize=12)
ax2016N.annotate('2016',xy=[.5,1.18],xycoords='axes fraction',ha='center',fontsize=12)
ax2017N.annotate('2017',xy=[.5,1.18],xycoords='axes fraction',ha='center',fontsize=12)
ax2015N.set_title('SA (g/kg)')
ax2016N.set_title('SA (g/kg)')
ax2017N.set_title('SA (g/kg)')
ax2015Si.set_title('$\Theta$ ($^{\circ}$C)')
ax2016Si.set_title('$\Theta$ ($^{\circ}$C)')
ax2017Si.set_title('$\Theta$ ($^{\circ}$C)')

fig.savefig('/data/eolson/results/MEOPAR/biomodelevalpaper/figsEval/EvalTS.eps',dpi=200,transparent=True)

In [ ]:
plt.hist(dataPSF.Z,50);